In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import re

In [2]:
# try debugging this later
# from crpapi import CRP, CRPApiError

In [3]:
# read in your api key
with open('crp_api_key.json') as f:
    secret = json.load(f)

In [4]:
def get_ids_for_orgs(org_string, api_key):
    """
    Takes in a partial string of an organization's name, 
    and saves a dataframe of all organizations containing the string
    along with each organization's ID. 
    
    Also need to pass in your CRP API key.
    """
    url = f'https://www.opensecrets.org/api/?method=getOrgs&org={org_string}&apikey=' + api_key
    resp = requests.get(url)
    print(resp.status_code)
    soup = BeautifulSoup(resp.content, 'html.parser')
    org_ids = []
    org_names = []

    for org in soup.find_all('organization'):
        org_ids.append(org['orgid'])
        org_names.append(org['orgname'].strip())
        
    df = pd.DataFrame({'org_id':org_ids, 'org_names':org_names})  # change to org_name!!!
    filepath = 'data/crp_ids_' + org_string + '.csv'
    df.to_csv(filepath, index=False)
    print('Saved file', filepath)

In [5]:
# get_ids_for_orgs('Int', secret['api_key'])

In [6]:
df = pd.read_csv('data/crp_ids_Int.csv')
print(df.shape)
df.head(30)

(300, 2)


,org_id,org_names
0,D000021803,ACA International
1,D000065572,ACORN International
2,D000046437,Advent International
3,D000071358,Advocare International
4,D000046547,Airports Council International
5,D000031360,Airtech International
6,D000046619,Albany International
7,D000031477,American Defense International
8,D000023974,American Intellectual Property Law Assn
9,D000000123,American International Group


Thoughts:

- 20,000 (daily?) API limit of `getOrgs` calls
- 2,000 limit on `orgSummary` calls. Create other accounts w/ fake email addresses?
- Maybe we can pull org ids by partial strings? We'll get tons of duplicates, but can filter those out later
- Brainstorm other ways to group org names (i.e., first three letters?)
- Somehow filter down to contractors that we're most interested in?

#### Looping over CRP ids to scrape total contributions

In [7]:
contribs_all = pd.DataFrame()

for i, row in df.head(3).iterrows():    
    url = 'https://www.opensecrets.org/orgs/totals.php?id=' + row['org_id']
    resp = requests.get(url, headers = {'User-Agent': 'ua'})
    print(resp.status_code)
    
    tables = pd.read_html(resp.content)
    contribs = tables[0]
    contribs['org_id'] = row['org_id']
    contribs['org_name'] = row['org_names']  # FIX LATER (make singular)
    cols = list(contribs.columns.copy())
    cols.insert(0, cols.pop())
    cols.insert(0, cols.pop())
    contribs = contribs[cols]
    contribs_all = pd.concat([contribs_all, contribs])
    contribs_all = contribs_all.reset_index(drop=True)

200
200
200


In [8]:
print(contribs_all.shape)
contribs_all.head()

(40, 12)


,org_id,org_name,Cycle,Total,Democrats,Republicans,% to Dems,% to Repubs,Individuals,PACs,Soft (Indivs),Soft (Orgs)
0,D000021803,ACA International,2020,"$28,500",$0,"$28,500",0%,100%,"$1,000","$27,500",$0,$0
1,D000021803,ACA International,2018,"$207,000","$17,000","$190,000",8%,92%,$0,"$207,000",$0,$0
2,D000021803,ACA International,2016,"$244,250","$18,200","$226,050",8%,93%,"$1,250","$243,000",$0,$0
3,D000021803,ACA International,2014,"$215,250","$11,500","$198,750",5%,92%,$500,"$214,750",$0,$0
4,D000021803,ACA International,2012,"$256,967","$11,000","$245,967",4%,96%,"$3,217","$253,750",$0,$0


Clean up `contribs_all`

In [9]:
contribs_all.columns = ['org_id','org_name','cycle','total','dems','repubs','pct_to_dems','pct_to_repubs','indivs','pacs','soft_indivs','soft_orgs']
contribs_all = contribs_all[contribs_all.cycle.str.lower().str.startswith('t') == False]
contribs_all.cycle = contribs_all.cycle.astype(int)
contribs_all = contribs_all[contribs_all.cycle > 2005]
print(contribs_all.shape)
contribs_all.head()

(21, 12)


,org_id,org_name,cycle,total,dems,repubs,pct_to_dems,pct_to_repubs,indivs,pacs,soft_indivs,soft_orgs
0,D000021803,ACA International,2020,"$28,500",$0,"$28,500",0%,100%,"$1,000","$27,500",$0,$0
1,D000021803,ACA International,2018,"$207,000","$17,000","$190,000",8%,92%,$0,"$207,000",$0,$0
2,D000021803,ACA International,2016,"$244,250","$18,200","$226,050",8%,93%,"$1,250","$243,000",$0,$0
3,D000021803,ACA International,2014,"$215,250","$11,500","$198,750",5%,92%,$500,"$214,750",$0,$0
4,D000021803,ACA International,2012,"$256,967","$11,000","$245,967",4%,96%,"$3,217","$253,750",$0,$0


#### Looping over CRP ids to scrape affiliates

Better to store the affiliates as a dictionary, rather than a df?

In [10]:
lm_df = pd.DataFrame({'org_id':['d000000104'], 'org_names':['Lockheed Martin']})
lm_df.head(3)

,org_id,org_names
0,d000000104,Lockheed Martin


In [11]:
affils_all = pd.DataFrame()

# for i, row in df.head(3).iterrows():  # test case: first 3
for i, row in lm_df.head(3).iterrows():  # test case: lockheed
    print(row['org_id'])
    for year in reversed(range(2006, 2019, 2)):

        url = 'https://www.opensecrets.org/orgs/totals.php?id=' + row['org_id'] + '&type=P&cycle=' + str(year)
        resp = requests.get(url, headers = {'User-Agent': 'ua'})
        print(resp.status_code)

        tables = pd.read_html(resp.content)

        if len(tables) > 1:  # affiliates table exists
            affils = tables[1]
            affils['org_id'] = row['org_id']
            affils['org_name'] = row['org_names']  # FIX LATER (make singular)
            affils['cycle'] = year
            cols = list(affils.columns.copy())
            cols.insert(0, cols.pop())
            cols.insert(0, cols.pop())
            cols.insert(0, cols.pop())
            affils = affils[cols]
            affils_all = pd.concat([affils_all, affils])
            affils_all = affils_all.reset_index(drop=True)

d000000104
200
200
200
200
200
200
200


In [13]:
print(affils_all.shape)
affils_all.head()

(45, 11)


,org_id,org_name,cycle,Affiliate,Total,Dems,Repubs,% to Dems,% to Repubs,Indivs,PACs
0,d000000104,Lockheed Martin,2018,Lockheed Fort Worth Co,$388,$0,$388,0%,100%,$388,$0
1,d000000104,Lockheed Martin,2018,Lockheed Martin Aeronautics,"$22,066","$1,306","$19,970",6%,91%,"$22,066",$0
2,d000000104,Lockheed Martin,2018,Lockheed Martin Information Technology,$480,$330,$150,69%,31%,$480,$0
3,d000000104,Lockheed Martin,2018,Lockheed Martin Space Systems,$706,$396,$310,56%,44%,$706,$0
4,d000000104,Lockheed Martin,2018,Martin Marietta Corp,$700,$0,$700,0%,100%,$700,$0
5,d000000104,Lockheed Martin,2018,Savi Technology,$610,$610,$0,100%,0%,$610,$0
6,d000000104,Lockheed Martin,2018,Sikorsky Aircraft,"$4,457","$1,416","$2,899",32%,65%,"$4,457",$0
7,d000000104,Lockheed Martin,2016,Lockheed Martin Aeronautics,"$46,870","$6,090","$40,613",13%,87%,"$46,870",$0
8,d000000104,Lockheed Martin,2016,Lockheed Martin Information Technology,$250,$0,$250,0%,100%,$250,$0
9,d000000104,Lockheed Martin,2016,Lockheed Martin Space Systems,"$2,883",$781,"$1,640",27%,57%,"$2,421",$0


Clean up `affils_all`

In [14]:
affils_all.columns = ['org_id','org_name','cycle','affiliate','total','dems','repubs','pct_to_dems','pct_to_repubs','indivs','pacs']
print(affils_all.shape)
affils_all

(45, 11)


,org_id,org_name,cycle,affiliate,total,dems,repubs,pct_to_dems,pct_to_repubs,indivs,pacs
0,d000000104,Lockheed Martin,2018,Lockheed Fort Worth Co,$388,$0,$388,0%,100%,$388,$0
1,d000000104,Lockheed Martin,2018,Lockheed Martin Aeronautics,"$22,066","$1,306","$19,970",6%,91%,"$22,066",$0
2,d000000104,Lockheed Martin,2018,Lockheed Martin Information Technology,$480,$330,$150,69%,31%,$480,$0
3,d000000104,Lockheed Martin,2018,Lockheed Martin Space Systems,$706,$396,$310,56%,44%,$706,$0
4,d000000104,Lockheed Martin,2018,Martin Marietta Corp,$700,$0,$700,0%,100%,$700,$0
5,d000000104,Lockheed Martin,2018,Savi Technology,$610,$610,$0,100%,0%,$610,$0
6,d000000104,Lockheed Martin,2018,Sikorsky Aircraft,"$4,457","$1,416","$2,899",32%,65%,"$4,457",$0
7,d000000104,Lockheed Martin,2016,Lockheed Martin Aeronautics,"$46,870","$6,090","$40,613",13%,87%,"$46,870",$0
8,d000000104,Lockheed Martin,2016,Lockheed Martin Information Technology,$250,$0,$250,0%,100%,$250,$0
9,d000000104,Lockheed Martin,2016,Lockheed Martin Space Systems,"$2,883",$781,"$1,640",27%,57%,"$2,421",$0


#### Exploring groupings of contractors names (from USA Spending)

In [ ]:
contractors_2018 = pd.read_csv('flatiron_mod3proj/contractors2018_ALL.csv')
print(contractors_2018.shape)
contractors_2018.head()

In [ ]:
c = contractors_2018.copy()
c.columns = ['idx','contractor_name']
c['prefix'] = c.contractor_name.str[:3].astype(str)
c['prefix_len'] = c.prefix.apply(lambda x: len(x))
# c = c[c.prefix_len == 3]

c.groupby('prefix').size().sort_values(ascending=False).head(10)